In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 19:42:22.855470: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:42:22.886154: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:42:22.887067: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 19:42:23.848266: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
290


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5039                 219.695  -     16   +     17       False
background_yield_0.5039             270.292  -     17   +     18       False
frac_Mbc_bkg1_0.5039               0.220183  -    0.1   +    0.1       False
frac_deltaE_bkg1_0.5039           -0.247011  -   0.19   +    0.2       False
True
Iteration 1 - Signal Yield: 219.69508378614108 ± 16.21671976047273
Running iteration 2/300...
200
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2985                 229.136  -     16   +     17       False
background_yield_0.2985             260.732  -     17   +     18       False
frac_Mbc_bkg1_0.2985               0.177792  -  0.099   +    0.1       False
frac_deltaE_bkg1_0.2985           -0.229863  -

## 150

In [3]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4732                 172.162  -     14   +     15       False
background_yield_0.4732             267.729  -     17   +     18       False
frac_Mbc_bkg1_0.4732               0.207683  -  0.098   +    0.1       False
frac_deltaE_bkg1_0.4732           -0.235255  -   0.19   +    0.2       False
True
Iteration 1 - Signal Yield: 172.16184716046396 ± 14.50422763415402
Running iteration 2/300...
150
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6336                 180.586  -     14   +     15       False
background_yield_0.6336             259.397  -     17   +     18       False
frac_Mbc_bkg1_0.6336               0.165802  -  0.097   +    0.1       False
frac_deltaE

## 100

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2896                 119.119  -     12   +     13       False
background_yield_0.2896             270.803  -     17   +     18       False
frac_Mbc_bkg1_0.2896               0.238064  -  0.099   +  0.099       False
frac_deltaE_bkg1_0.2896           -0.230033  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 119.11907536071064 ± 12.46248165031775
Running iteration 2/300...
100
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0043                 128.769  -     12   +     13       False
background_yield_0.0043             261.397  -     17   +     17       False
frac_Mbc_bkg1_0.0043               0.175744  -  0.096   +    0.1       False
frac_deltaE

## 75

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6991                 94.5672  -     11   +     12       False
background_yield_0.6991             270.355  -     17   +     18       False
frac_Mbc_bkg1_0.6991               0.234213  -  0.097   +  0.099       False
frac_deltaE_bkg1_0.6991           -0.226356  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 94.56719334533071 ± 11.301214432185585
Running iteration 2/300...
75
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9557                 105.775  -     11   +     12       False
background_yield_0.9557              259.27  -     17   +     17       False
frac_Mbc_bkg1_0.9557               0.168987  -  0.097   +  0.098       False
frac_deltaE_b

## 50

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8298                 70.6506  -    9.8   +     10       False
background_yield_0.8298             269.294  -     17   +     18       False
frac_Mbc_bkg1_0.8298               0.235694  -  0.097   +  0.097       False
frac_deltaE_bkg1_0.8298           -0.202893  -   0.19   +    0.2       False
True
Iteration 1 - Signal Yield: 70.65064582857373 ± 10.102673427703719
Running iteration 2/300...
50
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6236                 77.3033  -     10   +     11       False
background_yield_0.6236              262.58  -     17   +     18       False
frac_Mbc_bkg1_0.6236               0.194288  -  0.096   +  0.099       False
frac_deltaE_b

## 25

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
290


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5896                 42.5661  -    8.2   +    8.8       False
background_yield_0.5896             272.421  -     17   +     18       False
frac_Mbc_bkg1_0.5896               0.262512  -  0.095   +  0.098       False
frac_deltaE_bkg1_0.5896            -0.16523  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 42.5660921452101 ± 8.52474517787293
Running iteration 2/300...
25
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2964                 48.2135  -    8.5   +    9.1       False
background_yield_0.2964             266.783  -     17   +     18       False
frac_Mbc_bkg1_0.2964               0.227519  -  0.095   +  0.098       False
frac_deltaE_bkg1_0.2964           -0.135014  -   0

## 10

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6089                 32.6597  -    7.2   +    8.2       False
background_yield_0.6089             266.883  -     16   +     18       False
frac_Mbc_bkg1_0.6089               0.231721  -  0.095   +  0.096       False
frac_deltaE_bkg1_0.6089           -0.137031  -   0.19   +    0.2       False
True
Iteration 1 - Signal Yield: 32.65974138014148 ± 7.666423012349982
Running iteration 2/300...
10
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6809                 31.7721  -    7.1   +    8.1       False
background_yield_0.6809             267.847  -     17   +     18       False
frac_Mbc_bkg1_0.6809               0.241666  -  0.095   +  0.096       False
frac_deltaE_bk

## 5

In [5]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6899                 26.3759  -    6.8   +    7.6       False
background_yield_0.6899             268.652  -     17   +     18       False
frac_Mbc_bkg1_0.6899               0.242983  -  0.094   +  0.097       False
frac_deltaE_bkg1_0.6899           -0.125838  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 26.37590451291023 ± 7.199591580553649
Running iteration 2/300...
5
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6641                 28.0952  -    6.8   +    7.6       False
background_yield_0.6641             266.839  -     17   +     17       False
frac_Mbc_bkg1_0.6641               0.231461  -  0.094   +  0.097       False
frac_deltaE_bkg1

## 2

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2931                   24.05  -    6.5   +    7.3       False
background_yield_0.2931             268.084  -     17   +     17       False
frac_Mbc_bkg1_0.2931               0.235936  -  0.094   +  0.095       False
frac_deltaE_bkg1_0.2931           -0.143433  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 24.049957856000372 ± 6.867110772139964
Running iteration 2/300...
2
290
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.152                 24.4815  -    6.5   +    7.3       False
background_yield_0.152             267.649  -     17   +     17       False
frac_Mbc_bkg1_0.152               0.235003  -  0.094   +  0.096       False
frac_deltaE_bkg1_0.1

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_ele_signal, mc_ele_only_signal, mc_ele_scf, mc_ele_bkg, mc_ele_signal_weight, mc_ele_only_signal_weight, mc_ele_scf_weight, mc_ele_bkg_weight = process_MC_data_combined(
        file_signal_ele_Bppip,file_bkg_ele_Bppip, 'ele', '521', 'pip', 'ee', 2, 'rec521', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'ele', 'signal', mc_ele_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'signal', mc_ele_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'scf', mc_ele_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'scf', mc_ele_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'ele', 'bkg', mc_ele_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'ele', 'bkg', mc_ele_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_ele_only_signal["Mbc"]
    Signal_deltaE = mc_ele_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_ele_scf["Mbc"], mc_ele_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_ele_scf["deltaE"], mc_ele_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg2_decay_ee_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg2_decay_ee_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
290


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.086                 21.6062  -    6.3   +    7.1       False
background_yield_0.086             268.478  -     17   +     17       False
frac_Mbc_bkg1_0.086               0.240784  -  0.094   +  0.095       False
frac_deltaE_bkg1_0.086           -0.136399  -   0.19   +   0.19       False
True
Iteration 1 - Signal Yield: 21.606202131455923 ± 6.652945023731739
Running iteration 2/300...
0
290
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9996                 21.6062  -    6.3   +    7.1       False
background_yield_0.9996             268.478  -     17   +     17       False
frac_Mbc_bkg1_0.9996               0.240784  -  0.094   +  0.095       False
frac_deltaE_bkg1_0.9996           -0.136399  -   0.19 